# Quick Start

#### 필수 라이브러리 설치 및 openai api key 등록

In [27]:
!pip install --upgrade --quiet langchain langchain-openai

In [28]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*****************************************************"

## PromptTemplate

In [29]:
from langchain.prompts import PromptTemplate

#템플릿 생성 및 테스트
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [30]:
from langchain.prompts import PromptTemplate

#템플릿 생성 
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

## ChatPromptTemplate

In [31]:
from langchain_core.prompts import ChatPromptTemplate

#채팅 탬플릿 샘플
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
#
messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

In [32]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

#채팅 탬플릿 샘플 2
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


## LCEL

In [33]:
#template 적용
prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val

StringPromptValue(text='Tell me a joke')

In [34]:
#prompt 텍스트 추출
prompt_val.to_string()

'Tell me a joke'

In [35]:
#프롬프트 메세지 형태로 추출
prompt_val.to_messages()

[HumanMessage(content='Tell me a joke')]

In [36]:
#프롬프트 대화 형태로 추출 
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})

In [37]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

In [38]:
#텍스트로 채팅 추출
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."

### RunnableThrough

In [39]:
from langchain_core.runnables import RunnablePassthrough

#key를 미리 정의하여 value만으로 동작하도록 함
runnable_template = {"n": RunnablePassthrough()} | prompt_template
runnable_template.invoke(5)

StringPromptValue(text='Tell me a joke')

## String Prompt Composition

In [40]:
from langchain.prompts import PromptTemplate

#다중 변수 프롬프트
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

In [41]:
prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [42]:
#프롬프트 포맷 확인
prompt.format(topic="sports", language="korean")

'Tell me a joke about sports, make it funny\n\nand in korean'

In [43]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

chain = LLMChain(llm=model, prompt=prompt)

#프롬프트 실행
chain.run(topic="sports", language="korean")

'왜 농구 선수는 항상 춥죠? \n\n공이 계속 튀니까!'

## Chat prompt composition

In [44]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#명령 프롬프트
prompt = SystemMessage(content="You are a nice pirate")

#HumanMessage : 사용자 질의
#AIMessage : 답변
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

#input : User Message = HumanMessage
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

In [45]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

chain = LLMChain(llm=model, prompt=new_prompt)

chain.run("i said hi")

'Ah, my apologies. Hello there! How can I assist you today?'

## partial_variables

In [46]:
from datetime import datetime

#날짜 가져오는 함수 생성

def get_today():
    now = datetime.now()
    return now.strftime("%B %d")

get_today()

'March 18'

In [47]:
#함수 값을 전달
prompt_template = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요.",
    input_variables=["n"],
    partial_variables={"today": get_today},  # partial_variables에 함수를 전달
)

prompt_template.format(n=5)

'오늘의 날짜는 March 18 입니다. 오늘이 생일인 유명인 5명을 나열해 주세요.'

In [49]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

chain = LLMChain(llm=model, prompt=prompt_template)

#테스트
chain.run(n=5)

"1. 아놀드 슈워제네거 (Arnold Schwarzenegger) - 오스트리아 출신의 배우 및 정치인\n2. 아담 리바인 (Adam Levine) - 미국의 음악가 및 가수, 마룬 5의 보컬\n3. 루시 헤일 (Lucy Hale) - 미국의 배우 및 가수, '플리티드'의 주연으로 유명\n4. 퀸 라티파 (Queen Latifah) - 미국의 배우 및 래퍼\n5. 더블에스301 (SS301) - 대한민국의 보이 그룹, SS501의 유닛 그룹"